In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from tensorflow import feature_column

In [2]:
# upload theorem CSV
from google.colab import files
uploaded = files.upload()

Saving thms.csv to thms.csv


In [0]:
# load the theorems, provided the uploaded file was called "thms.csv"
import io

BATCH_SIZE = 64

thm_df = tf.data.experimental.make_csv_dataset(
    'thms.csv', BATCH_SIZE, label_name='is_proven')

stmts_df = pd.read_csv('thms.csv')['stmt']

In [0]:
# train test split

TEST_SIZE = 500
test_df = thm_df.take(TEST_SIZE)
train_df = thm_df.skip(TEST_SIZE)


In [0]:
feature_cols = []

feature_cols.append(feature_column.numeric_column('time_agg'))
feature_cols.append(feature_column.numeric_column('max_mem_agg'))
feature_cols.append(feature_column.numeric_column('num_exp_agg'))
feature_cols.append(feature_column.numeric_column('num_attempts'))

stmt_col = feature_column.embedding_column(
    feature_column.sequence_categorical_column_with_vocabulary_list('stmt',
    ('(', ')', ',', 'e', 'i', '*', 'x0', 'x1', 'x2', 'x3')), dimension=8)

In [0]:
inputs_num = {
    colname : tf.keras.layers.Input(name=colname, shape=(), dtype='float32') \
          for colname in ['time_agg', 'max_mem_agg', 'num_exp_agg', 'num_attempts']
}
inputs_txt = {
    'stmt' : tf.keras.layers.Input(name='stmt', shape=(), dtype='string')
}
all_inputs = dict(inputs_num)
all_inputs.update(inputs_txt)

features_num = tf.keras.layers.DenseFeatures(feature_cols)(inputs_num)
features_txt, features_txt_len = tf.keras.experimental.SequenceFeatures([stmt_col])(inputs_txt)
features_txt_len_msk = tf.sequence_mask(features_txt_len)

txt_node = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(features_txt, mask=features_txt_len_msk)

final_layer = tf.keras.layers.concatenate([features_num, txt_node])
final_layer = tf.keras.layers.Dense(64, activation='relu')(final_layer)
final_layer = tf.keras.layers.Dense(1)(final_layer)

model = tf.keras.Model(inputs=all_inputs,
              outputs=final_layer)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [101]:
model.fit(train_df, epochs=10,
          validation_data=test_df, 
          validation_steps=30, steps_per_epoch=1000)

Epoch 1/10
1000/1000 [==============================] - 7s 7ms/step - loss: 6.1590 - accuracy: 0.7764 - val_loss: 5.2386 - val_accuracy: 0.2953
Epoch 2/10
1000/1000 [==============================] - 5s 5ms/step - loss: 4.3435 - accuracy: 0.8135 - val_loss: 0.5488 - val_accuracy: 0.8823
Epoch 3/10
1000/1000 [==============================] - 5s 5ms/step - loss: 3.7006 - accuracy: 0.8384 - val_loss: 2.6385 - val_accuracy: 0.9187
Epoch 4/10
1000/1000 [==============================] - 5s 5ms/step - loss: 2.9845 - accuracy: 0.8726 - val_loss: 1.8382 - val_accuracy: 0.9229
Epoch 5/10
1000/1000 [==============================] - 5s 5ms/step - loss: 3.7718 - accuracy: 0.8641 - val_loss: 3.8416 - val_accuracy: 0.9214
Epoch 6/10
1000/1000 [==============================] - 5s 5ms/step - loss: 3.7369 - accuracy: 0.8676 - val_loss: 1.1213 - val_accuracy: 0.9161
Epoch 7/10
1000/1000 [==============================] - 5s 5ms/step - loss: 3.1354 - accuracy: 0.8780 - val_loss: 0.6692 - val_accuracy:

In [102]:
test_loss, test_acc = model.evaluate(test_df)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

500/500 [==============================] - 1s 2ms/step - loss: 4.1555 - accuracy: 0.9281
Test Loss: 4.15548849105835
Test Accuracy: 0.9280937314033508
